<a href="https://colab.research.google.com/github/ggvaldez/LGPD-Script/blob/main/LGPD_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para Rodar todo o script:

1 - defina a URL abaixo 

2 - clique em "Ambiente de Execução" e em seguida "Executar Tudo" (Ou pressione CRTL +F9)

-----------------------

Defina aqui a URL da planilha a ser utilizada para a LGPD

In [1]:
url_google_sheets = 'https://docs.google.com/spreadsheets/d/DOCUMENTO_AQUI' # Insira a url dentro das aspas

Autentica na sua conta Google e acessa a Planilha


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url(url_google_sheets)
sheet = wb.worksheet('2-Inventario')
values_list = sheet.col_values(3)

Identifica os Inventários e importa outras planilhas

Exibe Planilhas faltantes



In [ ]:
idp_sheets = {}
errors = []
in_header = True
for item in values_list:
  if (not(in_header)):
    try:
      idp_sheets[item] = wb.worksheet(item).get_all_values()
    except:
      errors.append("Falta a Planilha: " + item)
  elif (item == 'Nº Ref / ID'):
    in_header = False

print (errors)

In [ ]:
index = 0
for item in idp_sheets['ASCOM-01']:
  print(index, item)
  index = index + 1
